## Chapter 7: Data analysis

Data analysis is the process of exploring, understanding, and drawing insights from data. In this chapter, we will walk through essential steps in analyzing data, starting from examining individual variables (univariate analysis) to exploring relationships between multiple variables (multivariate analysis). We’ll also learn how to group similar data points using a technique called K-Means classification.

An important part of data analysis is curating the data—cleaning, selecting, and preparing it for meaningful insights. We began this process in the previous chapter, and we will continue doing it in the practical examples throughout this chapter.

## Univariate analysis

Univariate analysis is the simplest form of data analysis, where we examine one variable at a time. The goal is to understand the basic properties of that variable—such as its distribution, central tendency (like mean or median), and spread (like range or standard deviation). This type of analysis helps us get a clear picture of each variable on its own before looking at how variables relate to each other.

To illustrate this, we analyze a subset of wells from the Force 2020 ML competition, provided in the file [xeek_train_subset.csv](../data/xeek_train_subset.csv). We begin by loading the dataset along with [lith_colors.csv](../data/lith_colors.csv), which defines the color scheme for the lithologies. Using a dictionary, we translate lithology codes into lithology names and add a corresponding `LITH` column to the DataFrame. Finally, we use our `plot_utilities` package to generate a dictionary mapping each lithology to its associated color.

In [ ]:
# Import libraries required for the notebook
import os # import os to work with directories
import pandas as pd # import pandas as pd
import matplotlib.pyplot as plt # import matplotlib.pyplot as plt
import seaborn as sns # import seaborn as sns (for statistical data visualization)
import plot_utilities as pu # import our plot utilities package
import data_analysis as da # import our data analysis package

In [ ]:
# paths to files
path_1 = os.path.join("..", "data", "xeek_train_subset.csv") 
path_2 = os.path.join("..", "data", "lith_colors.csv")

# read the files into pandas dataframes
df_1 = pd.read_csv(path_1) # well logs
df_2 = pd.read_csv(path_2) # facies color codes

# make a dictionary of lithology numbers and lithology names
lithology_dict = {30000: 'Sandstone', 65030: 'SS/Shale',
                 65000: 'Shale', 80000: 'Marl', 
                 74000: 'Dolomite', 70000: 'Limestone', 
                 70032: 'Chalk', 88000: 'Halite', 
                 86000: 'Anhydrite', 99000: 'Tuff',
                 90000: 'Coal', 93000: 'Basement'}

# add lithology names to the dataframe, based on the second
# last column (which contains lithology numbers)
df_1["LITH"] = df_1[df_1.columns[-2]].map(lithology_dict)

# get the lithology colors
colors = pu.get_colors(df_1, df_2, int_col="LITH")
print(colors)

Let's analyze one variable in the dataset, for example gamma ray (`GR`). To begin, we can create a table where each column represents a lithology, and each row shows a statistical summary of `GR` values within that lithology—including the count, minimum, maximum, mean, standard deviation, and key percentiles. The function `property_table()` in the [analysis](data_analysis/analysis.py) module of our `data_analysis` package, generates this summary.

We can use this function to generate the statistical summary of GR in the different lithologies:

In [ ]:
# GR statistics for the different lithologies
table = da.property_table(df_1, "LITH", "GR")
table.head(10)

While this summary table is useful, a graphical display can make it easier to understand the distribution of `GR` values. The `property_plot()` function from our [analysis](data_analysis/analysis.py) module plots the property (`GR`) for each class (`LITH`) using either box plots or violin plots. Note that this and several functions in this chapter rely on the [Seaborn](https://seaborn.pydata.org) library (imported as `sns`), which provides a high-level interface for creating statistical graphics in Python.

Let's use this function to visualize the distribution of `GR` in the different lithologies using a box plot:

In [ ]:
# GR in the different lithologies as box plot
fig = da.property_plot(df_1, "LITH", colors, "GR")

The box plot is a graphical tool that shows how values are spread out in a dataset. It highlights the median (the middle value), the range of most values (the interquartile range), and any values that are unusually high or low—these are called *outliers*.

As shown in the plot, `GR` values exceeding 300 API are rare and likely represent spurious measurements. Let's remove these outliers from the dataset:

In [ ]:
# remove GR values > 300 API
df_1 = df_1[df_1["GR"] < 300]

# GR in the different lithologies as box plot
fig = da.property_plot(df_1, "LITH", colors, "GR")

We can also visualize the `GR` distribution using a violin plot. The following line generates this plot:

In [ ]:
# GR distribution as violin plot
fig = da.property_plot(df_1, "LITH", colors, "GR", type="violin")

The violin plot is a graphical representation that combines aspects of a box plot and a kernel density plot, showing the distribution of the variable, its density at different values, and its summary statistics, such as the median and quartiles.

## Multivariate analysis

We now move on to multivariate analysis, where we examine relationships between multiple variables. To start, we’ll explore the correlation between two variables using a cross plot—a simple yet powerful tool for identifying trends, patterns, and potential outliers.

The function `cross_plot` from our [analysis](data_analysis/analysis.py) module generates a cross plot of two variables and fits a linear trend line using SciPy’s `linregress` method, which performs a least-squares regression.

Let’s use this function to cross plot neutron porosity (`NPHI`) versus density (`RHOB`) for all the wells, with the data points colored by lithology:

In [ ]:
# cross plot NPHI vs RHOB
col_1, col_2 = "NPHI", "RHOB"

# create a figure and axis for the cross plot
fig, ax = plt.subplots()
        
# call the cross plot function
m, b, r_2 = da.cross_plot(df_1, col_1, col_2, "LITH", colors, ax)
ax.set_xlim(0, 1) # set x-axis limits (NPHI)
ax.set_ylim(1, 3) # set y-axis limits (RHOB)

# print the equation of the line and R² value
print(f"{col_1} = {m:.2f} {col_2} + {b:.2f}, R² = {r_2:.2f}")

plt.show()

We have now a predictive model for estimating neutron porosity from density and vice versa.

Using the Seaborn `FacetGrid()` method, it is possible to map the dataset onto multiple axes arrayed in a grid of rows and columns that correspond to levels (or classes) of variables in the dataset. For example, to draw the same cross plot above, but separated by well name (`WELL`), we can do the following:

In [ ]:
g = sns.FacetGrid(df_1, col='WELL', hue="LITH", palette=colors, col_wrap=4)
g.map(sns.scatterplot, 'NPHI', 'RHOB', marker="o", s=10, alpha=0.75)
g.set(xlim=(0, 1))
g.set(ylim=(1, 3))
g.add_legend(); # add semicolon to suppress the output

This tells us how the different lithologies vary in the wells, and how `NPHI` and `RHOB` are correlated in each well. 

Likewise, we can do the same plot with the data separated by groups (`GROUP`):

In [ ]:
g = sns.FacetGrid(df_1, col='GROUP', hue="LITH", palette=colors, col_wrap=4)
g.map(sns.scatterplot, 'NPHI', 'RHOB', marker="o", s=10, alpha=0.75)
g.set(xlim=(0, 1))
g.set(ylim=(1, 3))
g.add_legend(); # add semicolon to suppress the output

An interesting group is the Upper Permian Zechstein, which exhibits significant lithological variability. Halite, being nearly incompressible, shows no correlation between `NPHI` and `RHOB`, whereas the other lithologies—except possibly shale—display a negative correlation between these two variables.

Using Seaborn’s `pairplot()` method, it becomes straightforward to generate cross plots of multiple logs simultaneously. The code below cross plots four logs, with data points color-coded by lithology (`LITH`). Along the diagonal, since each variable cross plots against itself, a kernel density estimate (KDE) is shown.

In [ ]:
# make a smaller DataFrame of the wells,
# with the logs of interest
df_3 = df_1[["GR", "RHOB", "NPHI", "DTC", "LITH"]]

# cross plot the logs and color points
# by LITH
sns.pairplot(df_3, hue="LITH", 
                 palette=colors, 
                 diag_kind="kde"); # ; suppress the output

The covariance and correlation matrices quantify the relationships between log variables. The covariance matrix measures how pairs of variables vary together—positive values indicate they increase or decrease together, negative values mean they move in opposite directions, and values near zero suggest little or no linear relationship. Diagonal elements show the variance of each variable. The correlation matrix standardizes these values, showing how strongly variables are linearly related, and whether they are positively correlated (both increase together) or negatively correlated (one increases as the other decreases). The code below generates these matrices for the four selected logs:

In [ ]:
# drop LITH column
if "LITH" in df_3.columns:
    # drop the unit name
    df_3 = df_3.drop(columns=["LITH"])

# plot covariance and correlation matrices
cov = df_3.cov()
corr = df_3.corr()

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# plot covariance matrix
sns.heatmap(cov, annot=True, fmt=".2f", 
            cmap="coolwarm", vmin=-2,
            vmax=2, ax=axs[0])
axs[0].set_title("Covariance Matrix")

# plot correlation matrix
sns.heatmap(corr, annot=True, fmt=".2f", 
            cmap="coolwarm", vmin=-1, 
            vmax=1, ax=axs[1])
axs[1].set_title("Correlation Matrix")

fig.tight_layout()
plt.show()

## Clustering data

In subsurface studies, it can be helpful to group similar log responses to identify patterns or zones with shared properties—especially when dealing with large datasets. This is where classification comes in. Instead of manually interpreting each log trace, we can use computational methods to automatically group similar data points.

One simple and widely used technique for this is K-Means clustering, a type of unsupervised learning. Unlike supervised methods, K-Means doesn’t require labeled data—instead, it identifies clusters based solely on the structure of the data itself. The idea is to group data points so that those within the same group (or cluster) are more similar to each other than to those in other groups.

Thanks to the [scikit-learn](https://scikit-learn.org/stable/) library in Python, performing K-Means clustering is both straightforward and efficient. The function `optimal_clusters` from our [analysis](data_analysis/analysis.py) module, helps determine the optimal number of clusters for classifying the data—a crucial step, as selecting the appropriate number of clusters is often the most challenging part of the clustering process.

The function `cluster_data()` also from our [analysis](data_analysis/analysis.py) module, performs the actual clustering of the data using the K-Means algorithm.

In the cell below, we apply K-Means clustering to four selected logs from well 16/10-1 to group the data into distinct clusters. We start by selecting the logs—`GR`, `RHOB`, `NPHI`, and `DTC`—then remove any missing values (a crucial step for the algorithm to function properly). Next, we use our helper function to determine the optimal number of clusters for the classification.

In [ ]:
# well 16/10-1
df_well = df_1[df_1["WELL"] == "16/10-1"]

# key logs
df_4 = df_well[["DEPTH_MD", "GR", "RHOB", "NPHI", "DTC"]]

# drop NaNs
df_4 = df_4.dropna()

# data for clustering
data = df_4[["GR", "RHOB", "NPHI", "DTC"]]

# plot elbow curve
max_k = 10 # maximum number of clusters
fig = da.optimal_clusters(data, max_k)

The code produces an elbow plot and a silhouette score plot. Without going into details, these visualizations help estimate the optimal number of clusters: the elbow plot shows where adding more clusters no longer significantly reduces the within-cluster variance (typically at the "elbow" point), while the silhouette score plot indicates how well-separated the clusters are, with higher scores suggesting better-defined groups. Together, they guide us in selecting a number of clusters that balances simplicity with meaningful separation. For the selected log data, five clusters appear to provide a good balance.

The code below performs the clustering and generates cross plots of the logs, with data points color-coded according to their assigned cluster:

In [ ]:
# number of clusters
n_clusters = 5

# create a color palette for the clusters
palette = sns.color_palette("Set1", n_clusters)

# create a KMeans object
kmeans, labels = da.cluster_data(data, n_clusters)

# add the cluster labels to the DataFrame
df_4["cluster"] = labels

# plot the clusters
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# plot the clusters
sns.scatterplot(x="GR", y="RHOB", 
                hue="cluster", data=df_4, 
                palette=palette, ax=axs[0])
sns.scatterplot(x="NPHI", y="DTC",
                hue="cluster", data=df_4, 
                palette=palette, ax=axs[1])

# set the titles
axs[0].set_title("GR vs RHOB")
axs[1].set_title("NPHI vs DTC")

plt.show()

Finally, we plot the logs alongside their corresponding cluster assignments as a function of depth:

In [ ]:
# plot logs and clusters with depth
fig, axs = plt.subplots(1,5, figsize=(10, 10), sharey=True)

# label and invert the y axis
axs[0].set_ylabel("DEPTH (m)")
axs[0].invert_yaxis()

# plot logs
logs = ["GR", "RHOB", "NPHI", "DTC"]
for i, log in enumerate(logs):
    # plot the log as a blue curve
    axs[i].plot(df_4[log], df_4["DEPTH_MD"], '-', color="black", linewidth=0.5)
    # set the title
    axs[i].set_title(log)
    # set grid
    axs[i].grid()

# find the clusters tops
mask = df_4["cluster"] != df_4["cluster"].shift()
m_df = df_4[mask].reset_index(drop=True)

# plot the clusters
for i in range(len(m_df) - 1):
    # get the tops
    val1 = m_df.iloc[i]["DEPTH_MD"]
    val2 = m_df.iloc[i + 1]["DEPTH_MD"]
    # get the cluster
    cluster = m_df.iloc[i]["cluster"]
    # get the color
    color = palette[int(cluster)]
    # plot cluster as filled rectangle
    axs[-1].axhspan(val1, val2, facecolor=color)

axs[-1].set_title("Cluster")

fig.tight_layout()
plt.show()

The clusters shown may correspond to different lithologies, but not necessarily. They could also reflect uncertainties, noise, or artifacts introduced during the classification process. It is important to corroborate these results with additional information—such as core data, facies logs, or geological context—before drawing definitive conclusions.

As an example, you can compare the clusters log with the facies log in notebook [chapter6_2.ipynb](chapter6_2.ipynb).

## Exercises

At this point, you're mature enough in your understanding to begin designing your own exercises. Consider exploring questions that interest you, experimenting with other datasets, or extending the methods introduced here to new scenarios. You might revisit some examples—such as comparing cluster logs with facies logs—or dive deeper into the properties of specific lithologies across wells. The goal is to make the learning your own: ask questions, try things out, and don't be afraid to follow your inquiries.